In [79]:
import selenium
import requests as rq
import os
import bs4 #객체 읽기
import urllib.parse as up #한국어를 url로 바꾸기
import mysql.connector as sqlcnt #mysql 연결 -> 데이터 베이스
import datetime #검색한 날짜 입력하기 위해
import pytz # 서울 표준시로 바꾸기 위함

In [3]:
# 쿠팡 url
item,page=None,None
url=f'https://www.coupang.com/np/search?q={item}&channel=user&page={page}'
header={'Refer':""
        ,"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
        ,"Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"} # 이부분이 쿠팡은 추가로 필요함

In [2]:
#검색어를 url 형식에 맞게 encoding해줘야함
def encoder(x:str):
   return up.quote(x.encode('utf-8')) #우선 utf-8로 되어있어야함

def urlmkr(item,page):
   item=encoder(item)
   return f'https://www.coupang.com/np/search?q={item}&channel=user&page={page}'

In [8]:
# 1단계 : 객체 생성
# coupang_search=rq.Session()
# coupang_search.headers.update(header)

In [168]:
# 2단계  : url 수정 후 request
# 3단계 : 위의 틀을 기반으로 검색어를 자유롭게 바꿀수 있게끔 함수화 하기
def coupang(item):
    #해더
    header={'Refer':""
        ,"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
        ,"Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    def encoder(item:str):
       return up.quote(item.encode('utf-8'))
    def urlmkr(item,page):
        item=encoder(item)
        return f'https://www.coupang.com/np/search?q={item}&channel=user&page={page}'
    
    #객체 생성
    coupang_search=rq.Session()
    coupang_search.headers.update(header)
    
    #검색시작
    page=0 #시작 페이지 1부터
    items=[]
    empty=[]
    while True:
        page+=1
        url=urlmkr(item,page)
        try:
            search=coupang_search.get(url,timeout=5)
        # 검색 중지 조건 1. 모두 검색 완료 / 2. 페이지 오류-> 검색시간이 5초를 초과
            if search.status_code!=200:
                print(page)
                break
        except:
            break
        html=bs4.BeautifulSoup(search.text)
        # 판매목록 세부화
        items_html=html.select("li.search-product") #상품 하나씩 조회
        add_adress='www.coupang.com' #상품주소 에 추가해야 이동가능
        for sub_html in items_html:
            href=sub_html.select("a")[0].get_attribute_list("href")[0] #상품주소
            
            name=sub_html.select('div.name')[0].text.strip()   #상품명

            if sub_html.select("em.rating"): # 평점이 없을수도 있음
                rating=float(sub_html.select("em.rating")[0].text.strip()) #상품 평점

            if sub_html.select('span.rating-total-count'): #리뷰가 없을수도 있음
                count_review=sub_html.select('span.rating-total-count')[0].text.strip() #상품 리뷰수

            price=sub_html.select('strong.price-value')[0].text.strip() #상품가격
            
            # 검색한 시간을 기록하기 위해선(날짜가 변경돼 차후 다시 검색했을시 없을 수도 있으므로) 아래의 코드가 필요
            enc=pytz.timezone("Asia/Seoul")
            enc_time=datetime.datetime.now(enc) # 서울 표준시로 변경

            description={'item_address':add_adress+href,
                        'item_name':name,
                        'rating':rating,
                        'count-_reviews':count_review,
                        'price':price,
                        "search_time":f'{enc_time.year}-{enc_time.month}-{enc_time.day}'}
            items.append(description)
    return {"item":item,'result':items}

In [205]:
test1=coupang("제로콜라")

In [186]:
test1["result"][3]["count-_reviews"]

15097

In [171]:
# 4단계 MySql에 데이터베이스 업로드(선택사항)
import mysql

connection=mysql.connector.connect(
    host="localhost",
    user="root",
    password='1234',
    database='zero',
    port=3306)

cursor=connection.cursor()

In [193]:
"aa1".isnumeric()

False

In [200]:
def transnum(x:str):
    return int("".join([txt for txt in x if txt.isnumeric()]))

In [208]:
print(f'insert into coupang (keyward, {column_names}) valuse("{keyward}, {values}")')

insert into coupang (keyward, item_address, item_name, rating, count-_reviews, price, search_time) valuse("제로콜라, www.coupang.com/vp/products/2317120030?itemId=18320158241&vendorItemId=87722980686, 코카콜라 제로 캔, 355ml, 36개, 5.0, 52833, 53070, 2023-11-9")


In [206]:
# 데이터 추가
for data in test1["result"]: # 어떤 검색어로 검색했냐에 따라 달라질 예정
    keyward=test1["item"]
    # 쿼리문에서 열 이름 만들기
    column_names=str(list(data.keys()))[1:-2] #[] 지우기
    column_names=column_names.replace("'","") # '표시가 없어야 함

    #value 부분 만들기
    for key in data:
        #정수화
        if key in ['count-_reviews','price']:
            data[key]=transnum(data[key])
    values=str(list(data.values()))[1:-2]
    print(quary=f'insert into coupang (keyward, {column_names}) valuse("{keyward}, {values}")')

dict_values(['www.coupang.com/vp/products/7704502074?itemId=20636676557&vendorItemId=78549281206&sourceType=srp_product_ads&clickEventId=be112090-7ee8-11ee-8dc2-e9a565810c11&korePlacement=15&koreSubPlacement=1&clickEventId=be112090-7ee8-11ee-8dc2-e9a565810c11&korePlacement=15&koreSubPlacement=1', '부르르 제로 콜라, 12개, 1.5L', 5.0, 7790, 16900, '2023-11-9'])
dict_values(['www.coupang.com/vp/products/5071892418?itemId=6864726501&vendorItemId=74157319832', '펩시 제로슈거 라임향, 210ml, 30개입', 5.0, 150976, 14250, '2023-11-9'])
dict_values(['www.coupang.com/vp/products/5071892418?itemId=17909172379&vendorItemId=86532415890', '펩시 콜라 제로슈가 라임향, 355ml, 24개', 5.0, 150976, 15700, '2023-11-9'])
dict_values(['www.coupang.com/vp/products/5071892418?itemId=19643015891&vendorItemId=84639390218', '펩시 제로슈거 콜라, 210ml, 60개', 5.0, 150976, 24660, '2023-11-9'])
dict_values(['www.coupang.com/vp/products/6976298818?itemId=4093099810&vendorItemId=72077082112&sourceType=srp_product_ads&clickEventId=be112090-7ee8-11ee-897e-19b3

In [128]:
# 참고 구문
# mysql> creat table coupang (id int auto_increment primary key,
#     -> item_name varchar(255),
#     -> item_address varchar(255),
#     -> count_review int,
#     -> rating int,
#     -> price int,
#     -> search_time varchar(255))
#     -> ;
# mysql> alter table coupang add column keyward varchar(255);

# mysql> alter table coupang add unique item_address;

테이블 만들기 -> 검색어 저장 열 생성 및 주소에 unique 걸기(다른 검색어로 중복된 판매사이트가 나오면 안되므로)

In [119]:
test["result"][0]

{'item_adress': 'www.coupang.com/vp/products/7698998310?itemId=20608085096&vendorItemId=86170602196&sourceType=srp_product_ads&clickEventId=f176fec0-7ed7-11ee-9ac4-223cd49cc085&korePlacement=15&koreSubPlacement=1&clickEventId=f176fec0-7ed7-11ee-9ac4-223cd49cc085&korePlacement=15&koreSubPlacement=1',
 'name': '오리지널 코카콜라 190ml 1박스 30캔입 미니캔 캔콜라, 30개',
 'rating': '4.5',
 'count-_review': '(113)',
 'price': '24,000',
 'search_time': '2023-11-9'}